# CREACIÓN DE GRAFOS

A continuación se expone la forma en la que se crearon los grafos de conectividad entre electrodos. Para ello, tuvo que realizarse un paso de iteración de creación de archivos .edf sobre los dataloaders originales para poder proseguir con el cálculo de grafos. Por lo tanto, la primera parte de creación de datos .edf puede saltarse si se disponen de datos procesados en archivos .edf

## Importación de bibliotecas necesarias

In [5]:
import pickle
import eegraph
import mne
import pyedflib
import os

## Importación de dataloaders

In [6]:
train_loader_file = r'../dataloaders/mean_std_train'
val_loader_file = r'../dataloaders/mean_std_val.pkl'

with open(train_loader_file, 'rb') as file:
    dataloader_train = pickle.load(file)
    
with open(val_loader_file, 'rb') as file:
    dataloader_val = pickle.load(file)

for batch in dataloader_train:
    sample = batch
    break

## Creación de archivos .edf a partir de los dataloaders

In [ ]:
#Carpeta de destino
folder_path = r'C:\Users\hernandez\Documents\TFG\caueeg-dataset\graph_data'

for batch in val_loader:
    raw_data = batch['signal'].squeeze()  # Eliminar dimensiones adicionales
    n_channels = raw_data.shape[0]
    ch_names = [new_channel_names[i] for i in range(n_channels)]
    sfreq = 200
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

    # Crear el objeto RawArray con los datos procesados
    raw = mne.io.RawArray(raw_data, info)
    info = raw.info

    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    serial_number = batch['serial']
    file_name = 'signal_{}.fif'.format(serial_number)

    # COnfigurar correctamente el nombre del archivo
    file_name = file_name.replace('[', '').replace(']', '')
    file_name = file_name.replace("'", "")

    # Obtener la ruta completa del archivo .fif
    fif_file_path = os.path.join(folder_path, file_name)

    # Guardar el archivo .fif
    raw.save(fif_file_path, overwrite=True)

    # Convertir el archivo .fif a .edf utilizando pyEDFlib
    edf_file_path = fif_file_path.replace('.fif', '.edf')
    edf_file_path = edf_file_path.replace("'", "")

    with pyedflib.EdfWriter(edf_file_path, n_channels) as f:
        # Configurar la información de los canales
        for i in range(n_channels):
            channel_info = {
                'label': ch_names[i],
                'dimension': 'mV',
                'sample_rate': int(sfreq),
                'physical_max': raw_data[i].max(),
                'physical_min': raw_data[i].min(),
                'digital_max': int(raw_data[i].max()),
                'digital_min': int(raw_data[i].min())
            }
            f.setSignalHeader(i, channel_info)

        # Obtener los datos del archivo .fif
        raw_fif = mne.io.read_raw_fif(fif_file_path)
        data = raw_fif.get_data()

        # Escribir los datos en el archivo .edf
        f.writeSamples(data)

    print(f'Conversión completa para {file_name}')

    # Eliminar el archivo .fif
    os.remove(fif_file_path)

## Ejemplo de creación de grafos
Una vez se tiene una carpeta con los archivos .edf creados a partir de los dataloaders, se mostrará cómo crear grafos a través de un archivo de muestra

In [7]:
# Carga del archivo muestra
sample = '../Muestras/muestra2.edf'

In [12]:
#Creación de grafo vacío y carga de datos del archivo .edf en él
G = eegraph.Graph()
G.load_data(path=sample)

Extracting EDF parameters from C:\Users\hernandez\Documents\TFG\Untitled Folder\muestra2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 19
Sample rate: 200.0 Hz.
Duration: 9.995 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F7', 'T3', 'T5', 'F8', 'T4', 'T6', 'Fz', 'Cz', 'Pz']


In [13]:
window = 2 #Tamaño de ventana
connectivity_measure = 'pearson_correlation' # Se eligió calcular la conectividad por el coeficiente de Pearson

graphs, connectivity_matrix = G.modelate(window_size = window, connectivity = connectivity_measure, threshold=0.5)

Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 400.0), (400.0, 800.0), (800.0, 1200.0), (1200.0, 1600.0), (1600.0, 1999.0)]

Threshold: 0.5
Number of graphs created: 5


In [16]:
#Visualización de los grafos creados
G.visualize_html(graphs[0], 'graph1')